In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os, time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Initialization을 사용하는 방법: torch.nn.init

In [2]:
fc1 = nn.Linear(3,2)

### 각 모듈별로 weight나 bias객체 속성에 실제값이 들어있습니다.

In [3]:
[x for x in fc1.parameters()]

[Parameter containing:
 tensor([[ 0.2960, -0.4949, -0.4719],
         [ 0.4876, -0.0733,  0.0737]], requires_grad=True),
 Parameter containing:
 tensor([0.4311, 0.5641], requires_grad=True)]

In [4]:
fc1.weight

Parameter containing:
tensor([[ 0.2960, -0.4949, -0.4719],
        [ 0.4876, -0.0733,  0.0737]], requires_grad=True)

In [5]:
fc1.bias

Parameter containing:
tensor([0.4311, 0.5641], requires_grad=True)

### nn.init에는 여러가지 종류가 있습니다.

In [6]:
# 특정 분포에서 random하게 샘플링해서 초기화
nn.init.normal_(fc1.weight, mean=0.0, std=1.0)

# 특정 값으로 초기화
nn.init.zeros_(fc1.bias)
nn.init.constant_(fc1.bias, 0)

Parameter containing:
tensor([0., 0.], requires_grad=True)

In [7]:
fc1.weight

Parameter containing:
tensor([[ 0.5822,  1.8538, -2.3147],
        [ 0.7066, -0.1409,  0.6233]], requires_grad=True)

In [8]:
fc1.bias

Parameter containing:
tensor([0., 0.], requires_grad=True)

### 직접 값을 지정하는 것도 가능합니다.

In [9]:
tmp_tensor = torch.tensor([[1.,2.,3.],[4.,5.,6.]])

In [10]:
fc1.weight.data = tmp_tensor
fc1.weight

Parameter containing:
tensor([[1., 2., 3.],
        [4., 5., 6.]], requires_grad=True)

## Xavier Initialization & He Initialization

In [11]:
nn.init.xavier_normal_(fc1.weight)

Parameter containing:
tensor([[-0.2401, -0.5857, -0.4451],
        [-0.0358, -1.4800, -0.8516]], requires_grad=True)

In [12]:
nn.init.kaiming_normal_(fc1.weight)

Parameter containing:
tensor([[-0.2901, -0.5437,  0.8363],
        [-1.3419,  0.1662,  0.1981]], requires_grad=True)

# 실제 모델에 적용해봅시다

## 1. Model Class

In [13]:
class MyNet(nn.Module):
    def __init__(self, dim_in=784, dim_h1=50, dim_h2=100, dim_out=10):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(dim_in,dim_h1)
        self.fc2 = nn.Linear(dim_h1,dim_h2)
        self.fc3 = nn.Linear(dim_h2,dim_out)
        self.apply(self._init_weights) # 모델을 만들때, self._init_weights()를 그 즉시 호출하여 parameter 초기화
        
    def _init_weights(self, submodule):
        if isinstance(submodule, nn.Linear): # submodule이 nn.Linear에서 생성된 객체(혹은 인스턴스이면)
            nn.init.kaiming_normal_(submodule.weight) #해당 submodule의 weight는 He Initialization으로 초기화
            if submodule.bias is not None:
                submodule.bias.data.fill_(0.01) # 해당 submodule의 bias는 0.01로 초기화
            
    def forward(self, x):
        h1 = self.fc1(x)
        h1 = F.relu(h1)
        h2 = self.fc2(h1)
        h2 = F.relu(h2)
        out = self.fc3(h2)
        
        return out

In [14]:
# model의 module들은 뭐가 있는지 봅시다.
model = MyNet()
[x for x in model.modules()]

[MyNet(
   (fc1): Linear(in_features=784, out_features=50, bias=True)
   (fc2): Linear(in_features=50, out_features=100, bias=True)
   (fc3): Linear(in_features=100, out_features=10, bias=True)
 ),
 Linear(in_features=784, out_features=50, bias=True),
 Linear(in_features=50, out_features=100, bias=True),
 Linear(in_features=100, out_features=10, bias=True)]

## 2. train() 함수

- <span style = 'font-size:1.2em;line-height:1.5em'>`train()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 모델에 입력하기 적합하도록 x의 형태를 변경하고 원하는 device에 위치시키기</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 지난 batch로부터 계산했던 gradient를 초기화(`zero_grad()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Backpropagation으로 각 parameter의 gradient를 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> Gradient Descent로 parameter값 update</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> `trn_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 8.</b> 데이터 한 개당 평균 train loss 산출</span>

In [15]:
def train(model, data_loader, optimizer, criterion, device):
    model.train() # 모델을 학습모드로!
    trn_loss = 0
    for i, (x, y) in enumerate(data_loader):
        # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
        x = x.view(-1, 784).to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
        y = y.to(device)
        
        # Step 2. gradient 초기화
        optimizer.zero_grad()
        
        # Step 3. Forward Propagation
        y_pred_prob = model(x)
        
        # Step 4. Loss Calculation
        loss = criterion(y_pred_prob, y)
        
        # Step 5. Gradient Calculation (Backpropagation)
        loss.backward()
        
        # Step 6. Update Parameter (by Gradient Descent)
        optimizer.step()
        
        # Step 7. trn_loss 변수에 mini-batch loss를 누적해서 합산
        trn_loss += loss.item()
        
    # Step 8. 데이터 한 개당 평균 train loss
    avg_trn_loss = trn_loss / len(data_loader.dataset)
    return avg_trn_loss

## 3. evaluate()함수

- <span style = 'font-size:1.2em;line-height:1.5em'>`evaluate()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 모델에 입력하기 적합하도록 x의 형태를 변경하고 원하는 device에 위치시키기</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> 모델이 예측하는 레이블을 산출 (with `torch.argmax()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Minibatch의 실제 레이블(`y`)과 예측 레이블(`y_pred_label`)을 누적하여 저장</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> `eval_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> 데이터 한 개당 평균 evaluation loss와 accuracy 산출</span>

In [16]:
def evaluate(model, data_loader, optimizer, criterion, device):
    model.eval() # 모델을 평가모드로!
    eval_loss = 0
    
    results_pred = []
    results_real = []
    with torch.no_grad(): # evaluate()함수에는 단순 forward propagation만 할 뿐, gradient 계산 필요 X.
        for i, (x, y) in enumerate(data_loader):
            # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
            x = x.view(-1,784).to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
            y = y.to(device)

            # Step 2. Forward Propagation
            y_pred_prob = model(x)

            # Step 3. Loss Calculation
            loss = criterion(y_pred_prob, y)
            
            # Step 4. Predict label
            y_pred_label = torch.argmax(y_pred_prob, dim=1)
            
            # Step 5. Save real and predicte label
            results_pred.extend(y_pred_label.detach().cpu().numpy())
            results_real.extend(y.detach().cpu().numpy())
            
            # Step 6. eval_loss변수에 mini-batch loss를 누적해서 합산
            eval_loss += loss.item()

    # Step 7. 데이터 한 개당 평균 eval_loss와 accuracy구하기
    avg_eval_loss = eval_loss / len(data_loader.dataset)
    results_pred = np.array(results_pred)
    results_real = np.array(results_real)
    accuracy = np.sum(results_pred == results_real) / len(results_real)
    
    return avg_eval_loss, accuracy

## 4. 매 Epoch에 드는 시간 측정

In [17]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## 5. 학습하기

- <span style = 'font-size:1.2em;line-height:1.5em'>Dataset과 Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [18]:
# torchvision에서도 MNIST데이터를 제공합니다. 
# 이 데이터를 다운 받을 디렉토리(data_path) 존재 여부를 확인하고 존재하지 않으면 생성 
data_path = 'data'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
# data 변환 방법 선언 (data transform method)
# 아래 예시: numpy형태의 데이터를 받으면 걔를 tensor로 변환해줘!
transform = transforms.Compose([transforms.ToTensor()])

# dataset을 생성 (torchvision에서 제공하는 데이터를 다운 받고, 위의 방법대로 변환)
trn_dset = datasets.MNIST(root=data_path, train=True, transform=transform, download=True)
tst_dset = datasets.MNIST(root=data_path, train=False, transform=transform, download=True)

- <span style = 'font-size:1.2em;line-height:1.5em'>연산을 수행할 device를 설정하기</span>

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- <span style = 'font-size:1.2em;line-height:1.5em'>모델에 대한 객체 생성하기</span>

In [20]:
model = MyNet(dim_in=784, dim_h1=50, dim_h2=100, dim_out=10)
model = model.to(device)

- <span style = 'font-size:1.2em;line-height:1.5em'>학습한 모델을 저장할 directory 생성하기</span>

In [21]:
save_dir = 'models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

- <span style = 'font-size:1.2em;line-height:1.5em'>필요한 hyperparameter값 설정하기</span>

In [22]:
N_EPOCHS = 10
LR = 2e-4
BATCH_SIZE = 2**9

- <span style = 'font-size:1.2em;line-height:1.5em'>loss function 정의하기</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>예전 실습 파일에는 손실 함수인 F.nll_loss()가 train(), evaluate() 함수 안에서 바로 사용되었음</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>이번 손실 함수는 train(), evaluate() 함수 밖에서 정의하고 안에서는 정의한 함수를 사용하는 방식으로</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>어떻게 사용해도 상관없으니, 편한대로 사용하세요.</span>

In [23]:
loss_func = nn.CrossEntropyLoss(reduction='sum')

- <span style = 'font-size:1.2em;line-height:1.5em'>Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [24]:
trn_loader = DataLoader(trn_dset, batch_size = BATCH_SIZE, shuffle=True, drop_last=False)
tst_loader = DataLoader(tst_dset, batch_size = BATCH_SIZE, shuffle=False, drop_last=False)

- <span style = 'font-size:1.2em;line-height:1.5em'>optimizer 생성하기</span>

In [25]:
my_opt = optim.Adam(model.parameters(), lr = LR)

- <span style = 'font-size:1.2em;line-height:1.5em'>trn_data에 대해서 train()함수를, tst_data에 대해서 evaluate()함수를 반복적으로 호출하면서 모델을 학습</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>매 epoch마다 학습이 마무리되면, 모델 평가를 진행한다</span>

In [26]:
best_val_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    trn_loss = train(model=model, 
                     data_loader=trn_loader, 
                     optimizer=my_opt, 
                     criterion=loss_func,
                     device=device)
    val_loss, accuracy = evaluate(model=model, 
                                  data_loader=tst_loader, 
                                  optimizer=my_opt, 
                                  criterion=loss_func,
                                  device=device)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'{save_dir}/my_model2.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {trn_loss:.3f} | Test Loss: {val_loss:.3f} | Test Acc: {100*accuracy:.3f}% ')

Epoch: 01 | Time: 0m 12s
	Train Loss: 1.412 | Test Loss: 0.711 | Test Acc: 82.760% 
Epoch: 02 | Time: 0m 12s
	Train Loss: 0.540 | Test Loss: 0.406 | Test Acc: 89.480% 
Epoch: 03 | Time: 0m 12s
	Train Loss: 0.371 | Test Loss: 0.324 | Test Acc: 91.240% 
Epoch: 04 | Time: 0m 12s
	Train Loss: 0.310 | Test Loss: 0.282 | Test Acc: 92.180% 
Epoch: 05 | Time: 0m 12s
	Train Loss: 0.275 | Test Loss: 0.256 | Test Acc: 93.020% 
Epoch: 06 | Time: 0m 12s
	Train Loss: 0.251 | Test Loss: 0.237 | Test Acc: 93.290% 
Epoch: 07 | Time: 0m 12s
	Train Loss: 0.232 | Test Loss: 0.224 | Test Acc: 93.610% 
Epoch: 08 | Time: 0m 12s
	Train Loss: 0.216 | Test Loss: 0.209 | Test Acc: 93.950% 
Epoch: 09 | Time: 0m 12s
	Train Loss: 0.203 | Test Loss: 0.199 | Test Acc: 94.260% 
Epoch: 10 | Time: 0m 12s
	Train Loss: 0.192 | Test Loss: 0.189 | Test Acc: 94.620% 
